<a href="https://colab.research.google.com/github/bCBowers/sql_nba/blob/main/SQL_Workshop_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### SQL Workshop

***Environment and Data Setup***

Install needed libraries, ingest data, and setup SQL database

In [ ]:
pip install nba_api

In [ ]:
from nba_api.stats.endpoints import leagueleaders
import pandas as pd
from sqlalchemy import create_engine, text

# Pull data for the top 500 scorers
top_500_23_24 = leagueleaders.LeagueLeaders(
    season='2023-24',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:500]

engine = create_engine('sqlite://', echo=False)
conn = engine.connect()

top_500_23_24.to_sql(name='league_leaders_23_24', con=engine)

Can see the same data via both Pandas and SQL. Difference in visualization is due to this not being a native SQL environment.

In [ ]:
top_500_23_24.head(10)

In [ ]:
cursor = conn.execute(text("select * from league_leaders_23_24 limit 10"))
for row in cursor:
  print(row)

Load in additional years worth of data to enrich our analysis.

In [ ]:
top_500_22_23 = leagueleaders.LeagueLeaders(
    season='2022-23',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:500]

top_500_22_23.to_sql(name='league_leaders_22_23', con=engine)

top_500_21_22 = leagueleaders.LeagueLeaders(
    season='2021-22',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:500]

top_500_21_22.to_sql(name='league_leaders_21_22', con=engine)

top_500_20_21 = leagueleaders.LeagueLeaders(
    season='2020-21',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:500]

top_500_20_21.to_sql(name='league_leaders_20_21', con=engine)

top_500_19_20 = leagueleaders.LeagueLeaders(
    season='2019-20',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:500]

top_500_19_20.to_sql(name='league_leaders_19_20', con=engine)

Able to query both tables together using a UNION command.

In [ ]:
cursor = conn.execute(text("""
  with combined as
    (select *, '2023-24' as season from league_leaders_23_24
      union
    select *, '2022-23' as season from league_leaders_22_23
    )
    select * from combined
    limit 10
"""))
for row in cursor:
  print(row)

**Challenge Question**

Who's going to score the most points during the 2023-24, how many points will he score, and how many points will he average per game?

Notes:
- 2020-21 and 2021-22 seasons featured 72 games each. All other seasons features 82 games.
- Teams this season have played between 49 and 52 games.

In [ ]:
# Code to start with
cursor = conn.execute(text(
    """with combined as
    (
      select *, '2023-24' as season, 51 as total_games from league_leaders_23_24
      union
      select *, '2022-23' as season, 82 as total_games from league_leaders_22_23
      union
      select *, '2021-22' as season, 82 as total_games from league_leaders_21_22
      union
      select *, '2020-21' as season, 72 as total_games from league_leaders_20_21
      union
      select *, '2019-20' as season, 72 as total_games from league_leaders_19_20
    ),

    -- calculate the how many games each team has played this season and join back into main CTE
    max_games as
    (
      select
        team,
        max(gp) as games_played
      from
        league_leaders_23_24
      group by team
    ),
    new_combined as
    (
      select
        combined.*,
        coalesce(max_games.games_played, combined.total_games) as total_gp
      from
        combined left join
        max_games on combined.team=max_games.team and season='2023-24'
    )
    select * from new_combined
    limit 10
    """
))
for row in cursor:
  print(row)

My Solution

In [ ]:
cursor = conn.execute(text(
    """with combined as
    (
      select *, '2023-24' as season, 51 as total_games from league_leaders_23_24
      union
      select *, '2022-23' as season, 82 as total_games from league_leaders_22_23
      union
      select *, '2021-22' as season, 82 as total_games from league_leaders_21_22
      union
      select *, '2020-21' as season, 72 as total_games from league_leaders_20_21
      union
      select *, '2019-20' as season, 72 as total_games from league_leaders_19_20
    ),

    -- calculate the how many games each team has played this season and join back into main CTE
    max_games as
    (
      select
        team,
        max(gp) as games_played
      from
        league_leaders_23_24
      group by team
    ),
    new_combined as
    (
      select
        combined.*,
        coalesce(max_games.games_played, combined.total_games) as total_gp
      from
        combined left join
        max_games on combined.team=max_games.team and season='2023-24'
    ),

    -- find what percentage of games each player typically plays over the past five seasons
    perc_played_hist as
    (
      select
        *,
        cast(gp as float)/total_gp as perc_played
      from
        new_combined
    ),
    perc_played_player as
    (
      select
        player,
        player_id,
        avg(perc_played) as avg_perc_played
      from
        perc_played_hist
      group by 1,2
    ),

    -- join avg game played back into main CTE and use for final calculations
    current_year as
    (
      select
        new_combined.*,
        perc_played_player.avg_perc_played,
        cast(pts as float)/gp as ppg
      from
        new_combined inner join
        perc_played_player on new_combined.player_id=perc_played_player.player_id
      where new_combined.season='2023-24'
    ),
    final as
    (
      select
        player,
        player_id,
        pts + ppg*(82-total_gp)*avg_perc_played as total_pts,
        ppg,
        round(gp+(82-total_gp)*avg_perc_played) as games_played
      from current_year
    )

    select * from final
    order by total_pts desc
    limit 10
"""
))
for row in cursor:
  print(row)

Put any notes about your solution here.

This last line is to close the connection with the SQL database we created. If we run it before finishing, we need to recreate everything over again.

In [ ]:
conn.close()